In [156]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Resize, ToTensor, Normalize
import os
import glob
import PIL
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
%matplotlib inline

In [157]:
train_dir = "/opt/ml/input/data/train/"

In [158]:
data = pd.read_csv(os.path.join(train_dir, 'train.csv'))
image_dir = os.path.join(train_dir,'images')

print(image_dir)
print(data.dtypes)

/opt/ml/input/data/train/images
id        object
gender    object
race      object
age        int64
path      object
dtype: object


In [159]:
data.head()

,id,gender,race,age,path
0,000001,female,Asian,45,000001_female_Asian_45
1,000002,female,Asian,52,000002_female_Asian_52
2,000004,male,Asian,54,000004_male_Asian_54
3,000005,female,Asian,58,000005_female_Asian_58
4,000006,female,Asian,59,000006_female_Asian_59


In [160]:
image_dirs=[]
for img_path in data.path:
    image_dirs.append(image_dir+'/'+ img_path+'/')

print(image_dirs[0])

/opt/ml/input/data/train/images/000001_female_Asian_45/


In [161]:
image_paths=[]

for i_dir in image_dirs:
    image_paths.append(glob.glob(f'{i_dir}/**'))

tf = torchvision.transforms.ToTensor()
cnt=0

dataset_path=[]
dataset_label=[]

for img_path in image_paths:
    for image in img_path:
        dataset_path.append(image)
        if 'normal' in image:
            cnt+=1
            dataset_label.append(2)
        elif 'incorrect' in image:
            dataset_label.append(1)
        else:
            dataset_label.append(0)
            
        #img = PIL.Image.open(image)
        #plt.imshow(img)
        #plt.show()
        #img_t = tf(img)
        #print(img_t.size(), img_t[0][0][0]+img_t[1][0][0]+img_t[2][0][0])
        

In [162]:
cnt

2700

In [163]:
class TrainDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform
        
    def __getitem__(self, index):
        image = PIL.Image.open(self.img_paths[index])
        
        if self.transform:
            image = self.transform(image)
        return image
    
    def __len__(self):
        return len(self.img_paths)

In [176]:
transform = torchvision.transforms.Compose([Resize((512,384), PIL.Image.BILINEAR), ToTensor(), Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),])
dataset = TrainDataset(dataset_path, transform)

loader = DataLoader(dataset, shuffle=False)

In [177]:
class MyModel(nn.Module):
    def __init__(self, num_classes: int = 1000):
        super(MyModel, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(64, 32),
            nn.ReLU(inplace=True),
            nn.Linear(32, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [166]:
f = open(train_dir+"/isthisright.csv","w")
f.write("ImageID"+','+ "ans"+"\n")
for i in range(len(dataset_path)):
    f.write(dataset_path[i]+','+str(dataset_label[i])+'\n')
f.close()

In [167]:
submission = pd.read_csv(os.path.join(train_dir, 'isthisright.csv'))                                      

In [170]:
device = torch.device('cuda')
model = MyModel(num_classes=18).to(device)
model.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in loader:
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

submission.to_csv(os.path.join(train_dir, 'submission.csv'), index=False)
print('test inference is done!')

test inference is done!
